In [ ]:
import numpy as np
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import nltk

In [2]:
df = pd.read_csv('../data/pre-processed.csv')
df.head()

,index,label,preprocessed_news
0,0,fake,katia abreu diz vai colocar expulsao moldura n...
1,1,fake,ray peita bolsonaro conservador fake entrevist...
2,2,fake,reinaldo azevedo desmascarado policia federal ...
3,3,fake,relatorio assustador bndes mostra dinheiro pub...
4,4,fake,radialista americano fala sobre pt vendem ilus...


In [6]:
df['label'] = df.apply(lambda row: 1 if row.label == 'fake' else 0, axis=1)
df_true = df.loc[df['label'] == 0]
df_fake = df.loc[df['label'] == 1]

In [7]:
df_fake

,index,label,preprocessed_news
0,0,1,katia abreu diz vai colocar expulsao moldura n...
1,1,1,ray peita bolsonaro conservador fake entrevist...
2,2,1,reinaldo azevedo desmascarado policia federal ...
3,3,1,relatorio assustador bndes mostra dinheiro pub...
4,4,1,radialista americano fala sobre pt vendem ilus...
...,...,...,...
3595,3595,1,ft julgamento populista usou dinheiro publico ...
3596,3596,1,nota fiscal emitida petrobras mostra gasolina ...
3597,3597,1,estao estancando sangria bem debaixo nariz lem...
3598,3598,1,rj cidadao finge estar possuido pomba gira esc...


In [8]:
df_test = pd.concat([df_true.loc[df_true['index'] > 2000], df_fake.loc[df_fake['index'] < 3000]], ignore_index=True)
df_train = df

In [32]:
#Pre-processamento do texto para diminui


nltk.download('stopwords')

ps = PorterStemmer()


def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('portuguese')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gyova\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [33]:
df_train['preprocessed_news'] = df_train['preprocessed_news'].apply(stemming)


AttributeError: 'list' object has no attribute 'words'

In [34]:
X = df_train['preprocessed_news'].values
Y = df_train['label'].values


In [37]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)


<7200x79541 sparse matrix of type '<class 'numpy.float64'>'
	with 1705040 stored elements in Compressed Sparse Row format>

In [76]:
#LogisticRegression
lr_classifier = LogisticRegression()
lr_classifier.fit(x_train,y_train)

LogisticRegression()

In [79]:
x_train_pred = lr_classifier.predict(x_train)
training_data_acc = accuracy_score(x_train_pred,y_train)
print(f"Accuracy score: {training_data_acc}")
x_test_pred = lr_classifier.predict(x_test)
test_data_acc = accuracy_score(x_test_pred,y_test)
print(f"Accuracy score: {test_data_acc}")



Accuracy score: 0.9753703703703703
Accuracy score: 0.9372222222222222


In [80]:
y_pred = lr_classifier.predict(x_test_pred)

y_pred

#//LogisticRegression

ValueError: Expected 2D array, got 1D array instead:
array=[0 0 1 ... 0 0 1].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [85]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_sc = sc.fit_transform(x_train.toarray())
X_test_sc = sc.fit_transform(x_test.toarray())


array([[-0.0313474 ,  0.        , -0.01360954, ..., -0.03276684,
         0.        , -0.02328654],
       [-0.0313474 ,  0.        , -0.01360954, ..., -0.03276684,
         0.        , -0.02328654],
       [-0.0313474 ,  0.        , -0.01360954, ..., -0.03276684,
         0.        , -0.02328654],
       ...,
       [-0.0313474 ,  0.        , -0.01360954, ..., -0.03276684,
         0.        , -0.02328654],
       [-0.0313474 ,  0.        , -0.01360954, ..., -0.03276684,
         0.        , -0.02328654],
       [-0.0313474 ,  0.        , -0.01360954, ..., -0.03276684,
         0.        , -0.02328654]])

In [86]:
X_train_sc.shape[1,]

TypeError: tuple indices must be integers or slices, not tuple

In [84]:
from keras.models   import Sequential
from keras.layers import Dense

classifier = Sequential()
classifier.add(Dense(units=6,kernel_initializer="uniform", activation='relu',input_dim=3, input_shape=(X_train_sc.shape[1])))
classifier.add(Dense(units=6,kernel_initializer="uniform", activation='relu'))
classifier.add(Dense(units=6,kernel_initializer="uniform", activation='sigmoid'))
classifier.compile(optimizer="adam", loss='mae', metrics=['accuracy'])
classifier.fit(X_train_sc,y_train,batch_size=10, epochs=1)


TypeError: 'int' object is not iterable

In [73]:
y_pred = classifier.predict(X_test_sc)
y_pred = (y_pred > 0.5)
y_pred

57/57 [==============================] - 2s 41ms/step


array([[False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [ True,  True,  True,  True,  True,  True],
       ...,
       [False, False, False, False, False, False],
       [False, False, False, False, False, False],
       [ True,  True,  True,  True,  True,  True]])

In [74]:
from sklearn.metrics import  confusion_matrix
cm = confusion_matrix(y_test,y_pred)
cm

ValueError: Classification metrics can't handle a mix of binary and multilabel-indicator targets